In [1]:
# Copyright 2021 Google
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
import cirq
import recirq.time_crystals as time_crystals

In [3]:
import inspect
from IPython.display import Code
Code(inspect.getsource(time_crystals.symbolic_dtc_circuit_list), language='python')

def symbolic_dtc_circuit_list(
        qubits: Sequence[cirq.Qid],
        cycles: int
        ) -> List[cirq.Circuit]:

    """ Create a list of symbolically parameterized dtc circuits, with increasing cycles
    Args: 
        qubits: an ordered sequence of the available qubits, which are connected in a chain
        cycles: the maximum number of cycles, and the total number of resulting circuits
    Returns: 
        list of circuits with `0, 1, 2, ... cycles` many cycles
    """
     
    num_qubits = len(qubits)

    # Symbol for g
    g_value = sp.Symbol('g')

    # Symbols for random variance and initial state, one per qubit
    local_fields = sp.symbols('local_field_:' + str(num_qubits))
    initial_state = sp.symbols('initial_state_:' + str(num_qubits))

    # Symbols used for PhasedFsimGate, one for every qubit pair in the chain
    thetas = sp.symbols('theta_:' + str(num_qubits - 1))
    zetas = sp.symbols('zeta_:' + str(num_qubits - 1))
    chis = sp.symbols('chi_:' + str(num_qubits - 1))
    gammas = sp.symbols('gamma_:' + str(num_qubits - 1))
    phis = sp.symbols('phi_:' + str(num_qubits - 1))

    # Initial moment of Y gates, conditioned on initial state
    initial_operations = cirq.Moment([cirq.Y(qubit) ** initial_state[index] for index, qubit in enumerate(qubits)])

    # First component of U cycle, a moment of ZX gates. 
    sequence_operations = []
    for index, qubit in enumerate(qubits):
        sequence_operations.append(cirq.PhasedXZGate(
                x_exponent=g_value, axis_phase_exponent=0.0,
                z_exponent=local_fields[index])(qubit))

    # Begin U cycle
    u_cycle = [cirq.Moment(sequence_operations)]

    # Second and third components of U cycle, a chain of 2-qubit PhasedFSim gates
    #   The first component is all the 2-qubit PhasedFSim gates starting on even qubits
    #   The second component is the 2-qubit gates starting on odd qubits
    operation_list, other_operation_list = [],[]
    previous_qubit, previous_index = None, None
    for index, qubit in enumerate(qubits):
        if previous_qubit is None:
            previous_qubit, previous_index = qubit, index
            continue

        # Add an fsim gate 
        coupling_gate = cirq.ops.PhasedFSimGate(
            theta=thetas[previous_index], 
            zeta=zetas[previous_index],
            chi=chis[previous_index],
            gamma=gammas[previous_index],
            phi=phis[previous_index]
        )
        operation_list.append(coupling_gate.on(previous_qubit, qubit))
        
        # Swap the operation lists we're adding to, to avoid two-qubit gate overlap
        previous_qubit, previous_index = qubit, index
        operation_list, other_operation_list = other_operation_list, operation_list

    # Add the two components into the U cycle
    u_cycle.append(cirq.Moment(operation_list))
    u_cycle.append(cirq.Moment(other_operation_list))

    # Prepare a list of circuits, with n=0,1,2,3.... cycles
    circuit_list = []
    total_circuit = cirq.Circuit(initial_operations)
    circuit_list.append(total_circuit.copy())
    for c in range(cycles):
        for m in u_cycle:
            total_circuit.append(m)
        circuit_list.append(total_circuit.copy())

    return circuit_list

In [8]:
qubits = [cirq.GridQubit(0,i) for i in range(3)]
circuit_list = time_crystals.symbolic_dtc_circuit_list(qubits, 2)
for circuit in circuit_list: 
    print('circuit of length ' + str(len(circuit)))
    print(circuit)

circuit of length 1
(0, 0): ───Y^initial_state_0───

(0, 1): ───Y^initial_state_1───

(0, 2): ───Y^initial_state_2───
circuit of length 4
(0, 0): ───Y^initial_state_0───PhXZ(a=0,x=g,z=local_field_0)───PhFSim(theta_0, zeta_0, chi_0, gamma_0, phi_0)────────────────────────────────────────────────────
                                                               │
(0, 1): ───Y^initial_state_1───PhXZ(a=0,x=g,z=local_field_1)───PhFSim(theta_0, zeta_0, chi_0, gamma_0, phi_0)───PhFSim(theta_1, zeta_1, chi_1, gamma_1, phi_1)───
                                                                                                                │
(0, 2): ───Y^initial_state_2───PhXZ(a=0,x=g,z=local_field_2)────────────────────────────────────────────────────PhFSim(theta_1, zeta_1, chi_1, gamma_1, phi_1)───
circuit of length 7
(0, 0): ───Y^initial_state_0───PhXZ(a=0,x=g,z=local_field_0)───PhFSim(theta_0, zeta_0, chi_0, gamma_0, phi_0)────────────────────────────────────────────────────PhXZ(a=0,x=g,z=l